In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

In [4]:
df = pd.read_csv("../data/train_clean.csv")
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,label_count,clean_text,text_len
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0,explanation edits made username hardcore metal...,26
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,daww matches background colour im seemingly st...,11
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0,hey man im really trying edit war guy constant...,22
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0,cant make real suggestions improvement wondere...,49
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0,sir hero chance remember page thats,6


In [8]:
max_vocab_size = 100000
tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<OOV>")

df['clean_text'] = df['clean_text'].fillna("")
df['clean_text'] = df['clean_text'].astype(str)

tokenizer.fit_on_texts(df['clean_text'])

with open("../models/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [11]:
sequences = tokenizer.texts_to_sequences(df['clean_text'])

lengths = [len(seq) for seq in sequences]
print("90th percentile length: ", np.percentile(lengths,90))

max_len = 150
X = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

90th percentile length:  77.0


In [ ]:
## Target Labels
y = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

In [13]:
embedding_dim = 100
embedding_index = {}

with open("../data/glove.6B.100d.txt", encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs
        
print(f"Loaded {len(embedding_index)} word vectors from GloVe.")        

Loaded 400000 word vectors from GloVe.


In [14]:
## embedding Matrix
word_index = tokenizer.word_index
num_words = min(max_vocab_size, len(word_index) + 1)

embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i >= max_vocab_size:
        continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [16]:
## saving arrays
np.save("../data/X.npy", X)
np.save("../data/y.npy", y)
np.save("../data/embedding_matrix.npy", embedding_matrix)

NameError: name 'df_train' is not defined